# Rock Paper Scissors

In this tutorial you will learn how to build a Lamden smart contract for the classic game Rock, Paper, Scissors. Through this contract, two players can play Rock, Paper, Scissors over the Lamden Blockchain. 

This is an advanced example. Please make sure you have gone over the previous examples before this one so you have a better understanding of contracting as a whole. 

### Important Disclaimer:
This is an example smart contract. It is not production ready. It needs more tests. It also does not address certain timing edge cases. For example a user can just not reveal their choice and stall a game forever. The solution is left as an exercise for you. It is important when building a smart contract that you think of all edge cases and test heavily before deploying it to the blockchain. 

First let's import some things we will need later.

In [1]:
from contracting.stdlib.bridge.hashing import sha3
from contracting.client import ContractingClient
import secrets

Below is the function `rps_contract` containing the entire Rock, Paper, Scissors contract. It is never run as a Python function. Instead contracting extracts constructor, storage variables, public and private functions from it when it is submitted to the blockchain. We recomend reading on below the contract and then investigating the contract functions when they are being called.

In [2]:
def rps_contract():
    # This tells the blockchain to make space for data. Variable and Hash are not Python builtins. 
    # They are globals (globals are something you can access without importing it) made available by contracting. 
    # Calling Variable() creates a new piece of memory within the blockchain, that can hold a single value. 
    # If we say "foo = Variable()" then we can interact with that piece of memory through "foo". 
    # We can set the piece of memory to 1, for example by calling "foo.set(1)".
    # Setting the value will change the value for the next call to the smart contract.
    # We can get the piece of memory by calling "foo.get()".
    next_game_id = Variable()
    
    game_id_to_password_hash = Hash()
    
    game_id_to_player1_choice_hash = Hash()
    game_id_to_player2_choice_hash = Hash()
    
    game_id_to_player1_choice = Hash()
    game_id_to_player2_choice = Hash()
            
    # @construct means that this is the function that will be called ONCE when the smart contract is created
    @construct
    def constructor():
        # Game id starts at 0.
        next_game_id.set(0)
        
    # @export makes this function public, so it can be called by anyone on a deployed contract.
    # By calling start_game player1 creates a game and also submits their hashed and salted choice.
    @export
    def start_game(password_hash, player1_choice_hash):
        # This retrieves unique game Id from the blockchain. 
        unique_game_id = next_game_id.get()
        
        assert get_game_state(unique_game_id) == "game_doesnt_exist", "this is a bug in the contract. new game id already exists"
        
        # This increments the number by 1, so the next game has a unique Id.
        next_game_id.set(next_game_id.get() + 1)       
        
        # Hash throws error on integer keys. That's why we convert to a string
        # Remember player1 choice hash, and password hash for this game. 
        game_id_to_password_hash[str(unique_game_id)] = password_hash

        game_id_to_player1_choice_hash[str(unique_game_id)] = player1_choice_hash
        
        # assert get_game_state(unique_game_id) == "only_player1_submitted", "this is a bug in the contract. after starting a new game only player 1 must have submitted"
        
        return unique_game_id
    
    # By calling submit_choice player2 submits their hashed and salted choice.
    @export
    def submit_choice(game_id, game_password, player2_choice_hash):
        assert get_game_state(game_id) == "only_player1_submitted", "submit_choice can only be called if only player 1 has submitted their choice"

        # Check that this is the right password for the game.
        assert hashlib.sha3(game_password) == game_id_to_password_hash[str(game_id)], 'Wrong password!'
        
        # Remember player2's choice.
        game_id_to_player2_choice_hash[str(game_id)] = player2_choice_hash
        
        assert get_game_state(game_id) == "both_players_submitted", "this is a bug in the contract. after submitting player2 choice both players must have submitted"
        
        return
    
    # Returns 'player1_wins' if player1 is the winner.
    # Returns 'player2_wins' if player2 is the winner.
    # Returns 'tie' if both players made the same choice.
    # Returns 'game_doesnt_exist' if the game doesn't exist
    # Returns 'player1_has_submitted' if the game has been started and only player1 has submitted their choice
    # Returns 'both_players_have_submitted' if both players have submitted but none has revealed their choice
    # Returns 'only_player1_revealed' after player1 but not player2 has revealed their choice
    # Returns 'only_player2_revealed' after player2 but not player1 has revealed their choice
    
    @export
    def get_game_state(game_id):
        if next_game_id.get() <= game_id:
            return "game_doesnt_exist"
        
        player1_hashed_choice = game_id_to_player1_choice_hash[str(game_id)]
        player2_hashed_choice = game_id_to_player2_choice_hash[str(game_id)]

        if player1_hashed_choice is not None and player2_hashed_choice is None:
            return "only_player1_submitted"
        
        player1_choice = game_id_to_player1_choice[str(game_id)]
        player2_choice = game_id_to_player2_choice[str(game_id)]
              
        if player1_hashed_choice is not None and player2_hashed_choice is not None and player1_choice is None and player2_choice is None:
            return "both_players_submitted"
        
        assert player1_hashed_choice is not None and player2_hashed_choice is not None, "this is a bug in the contract. error code 1"
        # For the rest of the function we know that both players have submitted their choices 
        # and one player has revealed their choice
        
        if player1_choice is not None and player2_choice is None:
            return "only_player1_revealed"
        
        if player1_choice is None and player2_choice is not None:
            return "only_player2_revealed"
        
        # Make sure that both players have submitted their choices
        assert player1_choice is not None and player2_choice is not None, "this is a bug in the contract. error code 2"
        # For the rest of the function we know that both players have submitted their choices
        
        # Now that we have both choices we can resolve the game
        
        if player1_choice == player2_choice:
            return "tie"

        if beats(player1_choice, player2_choice):
            return "player1_wins"
        
        if beats(player2_choice, player1_choice):
            return "player2_wins"
        
    @export
    def is_valid_choice(choice):
        return choice in ["rock", "paper", "scissors"]
      
    # Returns whether choice1 beats choice2
    @export
    def beats(choice1, choice2):
        assert is_valid_choice(choice1), "choice1 must be a valid choice"
        assert is_valid_choice(choice2), "choice2 must be a valid choice"

        if choice1 == "rock" and choice2 == "scissors":
            return True
        
        if choice1 == "paper" and choice2 == "rock":
            return True
        
        if choice1 == "scissors" and choice2 == "paper":
            return True
        
        return False
    
    # By calling reveal a player can reveal their unhashed choice.
    # Player1 has to call the function with is_player1=true.
    # Player2 has to call the function with is_player1=false.
    @export
    def reveal(game_id, choice, choice_salt, is_player1):
        if is_player1:
            assert get_game_state(game_id) in ["both_players_submitted", "only_player2_revealed"], "reveal can only be called by player1 if no player or only player2 has revealed"
        else:
            assert get_game_state(game_id) in ["both_players_submitted", "only_player1_revealed"], "reveal can only be called by player2 if no player or only player1 has revealed"
        
        # Make sure players can only reveal valid choices
        assert is_valid_choice(choice), "choice must be rock, paper or scissors"
                
        # Now we need to check that the reveal hashes to what was submitted earlier
        
        salted_choice = choice + choice_salt
        hashed_choice = hashlib.sha3(salted_choice)
        
        if is_player1:
            assert game_id_to_player1_choice_hash[str(game_id)] == hashed_choice, "Player 1 has revealed a choice different from what they submitted"
        else:
            assert game_id_to_player2_choice_hash[str(game_id)] == hashed_choice, "Player 2 has revealed a choice different from what they submitted"
            
        # Now we're sure that the player has revealed the choice they have previously submitted the hashed salted version of
        
        # Remember the choice
        if is_player1:
            game_id_to_player1_choice[str(game_id)] = choice
        else:
            game_id_to_player2_choice[str(game_id)] = choice
        
        return

Welcome back! Lets get this contract into the blockchain. To interact with the blockchain we need a client. 

In [4]:
client = ContractingClient(signer='ren')

Get rid of all state of the blockchain so we have a blank slate. Otherwise running this script twice causes problems. 

In [5]:
client.flush()

Now we submit the contract to the blockchain. `client.submit` doesn't run the function `rps_contract` but examines it, and extracts its public functions, state variables, ...

In [6]:
client.submit(rps_contract)

Get a handle for the contract that we can interact with.

In [7]:
contract = client.get_contract('rps_contract')

With the contract in the blockchain we can now play Rock, Paper, Scissors. Our players for this example are Alice and Bob.

Alice (player 1) chooses single use password. Only the person that has the password can join the game and play with Alice. Everything that starts with alice_ is only visible to Alice. 

In [8]:
alice_game_password = "trollbridge"

Alice hashes the password so she can submit it to the blockchain without sharing the actual password. She does this because everything on the blockchain is public, and she wants only the person she chooses to play the game with to have the password. 

In [9]:
alice_game_password_hash = sha3(alice_game_password)
alice_game_password_hash

'47a5bcfb0d81053f5025ab57e6b94f43751f91bdb16fc0d63595223dc78ec1b4'

Alice chooses rock.

In [10]:
alice_choice = "rock"

We can't submit the choice to the blockchain as plain text, because then Bob (player 2) can see it and win by choosing paper. 

In [11]:
alice_choice_hash = sha3(alice_choice)
alice_choice_hash

'bd996e2dc82a7c3e2c2da41291648e852b62b01fb09bcb6ed77975c273f08404'

The problem with submitting Alices choice like this is the 3 choices will be hashed the same every time. Bob (player 2) can know what each of the hashes for the 3 choices and pick paper to win. To fix this Alice needs to pick a random salt to hash with her choice so that Bob can't guess her choice by looking at the hash.

In [12]:
alice_choice_salt = secrets.token_hex(32)
alice_choice_salt

'88ceef7a2c748432d5a150fcff5df717c8a67298365ae2e1969be4ee856ce39e'

Now we can combine alice_choice and alice_choice_salt and hash them together to create something that Bob can't guess Alices choice from. But Alice can later submit her choice and the salt to prove her choice.

In [13]:
alice_salted_choice = alice_choice + alice_choice_salt
alice_salted_choice

'rock88ceef7a2c748432d5a150fcff5df717c8a67298365ae2e1969be4ee856ce39e'

In [14]:
alice_salted_choice_hash = sha3(alice_salted_choice)
alice_salted_choice_hash

'dcbd95890db4648405b1e04541b6dcabacc1c3e958172171262dc11d6dacebf7'

Before a game is started it is in state `"game_doesnt_exist"`

In [15]:
assert contract.get_game_state(game_id=0) == "game_doesnt_exist"

Now Alice starts a game so she can invite Bob to play.

In [16]:
alice_game_id = contract.start_game(password_hash=alice_game_password_hash, player1_choice_hash=alice_salted_choice_hash) 
alice_game_id

0

Alice gets back a game Id.

The game is now in state "only_player1_submitted"

In [17]:
assert contract.get_game_state(game_id=alice_game_id) == "only_player1_submitted"

Now Alice has to tell Bob the password and the game Id. This could be done over a messenger or built into the frontend of an application. 

Everything that starts with bob_ is only visible to Bob.

In [18]:
bob_game_password = alice_game_password
bob_game_id = alice_game_id

Now it is Bobs turn.
Bob chooses scissors.

In [19]:
bob_choice = "scissors"

And now Bob has to salt his choice.

In [20]:
bob_choice_salt = secrets.token_hex(32)
bob_choice_salt

'13729fb6834f46b30046d82ba9e624821a479ed1f7714152c8a4da81b42d1213'

And now we combine bob_choice and bob_choice_salt together and hash them.

In [21]:
bob_salted_choice = bob_choice + bob_choice_salt
bob_salted_choice

'scissors13729fb6834f46b30046d82ba9e624821a479ed1f7714152c8a4da81b42d1213'

In [22]:
bob_salted_choice_hash = sha3(bob_salted_choice)
bob_salted_choice_hash

'9af37ec1317afab75c13f89f1d4a8b4c39f77ae52e7ad05116a5f8b2f9995125'

Bob now needs to submit his choice to the blockchain. Only Bob has the game password so only Bob can join Alices game.

In [23]:
contract.submit_choice(game_password=bob_game_password, game_id=bob_game_id, player2_choice_hash=bob_salted_choice_hash)

The game is now in state "both_players_submitted"

In [24]:
assert contract.get_game_state(game_id=bob_game_id) == "both_players_submitted"

Now that both players have submitted their hashed and salted choices, both players can reveal their choices.
The order doesn't matter. Alice goes first in this example.

In [25]:
contract.reveal(game_id=alice_game_id, choice=alice_choice, choice_salt=alice_choice_salt, is_player1=True)

The game is now in state "only_player1_revealed"

In [26]:
assert contract.get_game_state(game_id=alice_game_id) == "only_player1_revealed"

Bob goes second

In [27]:
contract.reveal(game_id=bob_game_id, choice=bob_choice, choice_salt=bob_choice_salt, is_player1=False)

Now we can see who won the game

In [28]:
contract.get_game_state(game_id=alice_game_id)

'player1_wins'

As expected Alice, who is player1, wins!

## tests

Test function is_valid_choice

In [29]:
assert contract.is_valid_choice(choice="rock")
assert contract.is_valid_choice(choice="paper")
assert contract.is_valid_choice(choice="scissors")
assert not contract.is_valid_choice(choice="airplane")

Test function beats

In [30]:
assert contract.beats(choice1="rock", choice2="scissors")
assert not contract.beats(choice1="rock", choice2="rock")
assert not contract.beats(choice1="rock", choice2="paper")

assert contract.beats(choice1="paper", choice2="rock")
assert not contract.beats(choice1="paper", choice2="paper")
assert not contract.beats(choice1="paper", choice2="scissors")

assert contract.beats(choice1="scissors", choice2="paper")
assert not contract.beats(choice1="scissors", choice2="scissors")
assert not contract.beats(choice1="scissors", choice2="rock")